In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lxml import etree
%aimport json
from pathlib import Path
%aimport re
from typing import List, Dict
%aimport os
from xml.sax.saxutils import escape

In [3]:
from data_processing.text_processing import get_text_from_file, set_working_directory, get_working_directory
from data_processing.text_processing import normalize_quotes

In [4]:
from data_processing.text_processing import write_text_to_file

In [5]:
from data_processing.gpt_processing import token_count, set_api_client, get_api_client, get_active_batches
from data_processing.gpt_processing import generate_messages, run_immediate_chat_process, create_jsonl_file_for_batch, start_batch
from data_processing.gpt_processing import get_completed_batches, get_batch_response


In [6]:
user_message_string_clean = """{text}"""

In [7]:
def user_wrap_function_clean(text_block):
    return user_message_string_clean.format(text=text_block)

In [8]:
system_message_clean = """
- You are a meticulous and consistent world expert at cleaning OCR-generated Vietnamese text. 
- You are cleaning text from a 1950's Buddhist Journal. 
IMPORTANT: You will receive one page of text listed by lines: do not modify the line structure.
- You will minimally modify the text to fix any OCR errors.
- You will add <section>, <title>, <author>, <p>, <li>, <ul>, <ol> tags (only) where appropriate in text, based on text layout and content. 
- Do not remove any content from the main body of the text.   
- Use patterns in the text blocks (given by page) to infer patterns in the text.
- Use the semantic meaning of the text to infer corrections—but make no semantic changes. 
- Add diacritical marks if they are missing or clearly inaccurate. 
- Do not change any proper names, except to add missing diacritical marks if the context is clear.  
IMPORTANT: Output the corrected text only with no comments or additional formatting marks of any kind.
"""

### testing gpt-4o-mini system message

You are an intelligent, meticulous, and consistent expert at cleaning OCR-generated Vietnamese text. 
The text will be given in XML, and the output text should be in matching XML. 
Your goal is to minimally modify the text to generate a cleaned version. 
Do not remove any text from the main content.  
Formatting markers at the beginning and end of the text can be adjusted or removed as needed for clarity. 
You can use the semantic meaning of the text to infer corrections—but make no semantic changes. 
You can also add diacritical marks if they are missing or clearly inaccurate and can be determined by context.
Do not change any proper names, except to add missing diacritical marks if the context is clear.  
This particular text has a title marker: "Phat Giao Viet Nam," and also a publishing mark near the end of each page of text. 
The publishing mark is something like "Tu Vien HUE QUANG" + "TRUNG TAM DICH THUAT HAN NOM" and is often incomplete.
Text corresponding to these marks (or part thereof) and page numbers can be omitted.
Output the corrected text only with no comments.

### Process files in sequence and generate cleaning batch JSON files:

In [14]:
def split_xml_pages(text):
    """
    Splits an XML document into individual pages based on <page> tags
    and returns the text content of each page, excluding the <page> tags.

    Parameters:
    - text (str): The XML document as a string.

    Returns:
    - List[str]: A list of strings, each representing the content of a page.
    """
    # Convert text to bytes
    text_bytes = text.encode("utf-8")
    
    try:
        # Parse the XML text into an element tree
        root = etree.fromstring(text_bytes)
    except etree.XMLSyntaxError as e:
        # Extract the line number and column number of the error
        line_number = e.lineno
        column_number = e.offset
        
        # Find the offending line in the original text
        lines = text.splitlines()
        error_line = lines[line_number - 1] if line_number - 1 < len(lines) else "Unknown line"
        
        print(f"XMLSyntaxError: {e}")
        print(f"Offending line {line_number}, column {column_number}: {error_line}")
        return []  # Return an empty list or handle the error as needed

    # Extract text content of each <page> element
    pages = [etree.tostring(page, encoding="unicode") for page in root.findall(".//page")]
    
    return pages

In [35]:
def split_xml_pages(text, page_groups=None):
    """
    Splits an XML document into individual pages based on <page> tags.
    Optionally groups pages together based on page_groups.

    Parameters:
    - text (str): The XML document as a string.
    - page_groups (list of tuples, optional): A list of tuples defining page ranges to group together.
                                              Each tuple is of the form (start_page, end_page), inclusive.

    Returns:
    - List[str]: A list of strings, where each element is a single page (if no groups) or a group of pages.
    """
    from lxml import etree

    # Parse the XML text into an element tree
    try:
        root = etree.fromstring(text.encode("utf-8"))
    except etree.XMLSyntaxError as e:
        # Handle parsing errors with helpful debugging information
        line_number = e.lineno
        column_number = e.offset
        lines = text.splitlines()
        error_line = lines[line_number - 1] if line_number - 1 < len(lines) else "Unknown line"
        print(f"XMLSyntaxError: {e}")
        print(f"Offending line {line_number}, column {column_number}: {error_line}")
        return []  # Return an empty list if parsing fails

    # Extract all pages as a list of strings
    pages = [
        (int(page.get("page")), etree.tostring(page, encoding="unicode"))
        for page in root.findall(".//page")
    ]
    
    # Sort pages by page number
    pages.sort(key=lambda x: x[0])

    # If no page_groups, return individual pages
    if not page_groups:
        return [content for _, content in pages]

    # Group pages based on page_groups
    grouped_pages = []
    for start, end in page_groups:
        group_content = ""
        for page_num, content in pages:
            if start <= page_num <= end:
                group_content += content
        if group_content:
            grouped_pages.append(group_content)

    return grouped_pages

In [10]:
processed_journals = "../../processed_journal_data"
batch_ouput_prefix = "./journal_cleaning_batches/"

In [11]:
ocr_xml = get_text_from_file("phat-giao-viet-nam-1956-28/full_OCR_text_phat-giao-viet-nam-1956-28.xml", processed_journals)

In [15]:
pages = split_xml_pages(ocr_xml)

In [16]:
pages

['<page page="1">\n    PHẬT GIÁO\nVIỆT-NAM\nNGUYỆT SAN\nSỐ 28 THÁNG 3 NĂM KỶ HỘI\nTHÁNG 4 29\nBỘ CÔNG HỘI PHẢI GIÁO CHỊU NẮN XUẤT LÀM\n  </page>\n  ',
 '<page page="2">\n    MUC LUC\nNhững ngày đại-lễ Buddha Jayanti - P.GV.N\nThông điệp của phái đoàn - P.G.V.N.\n  tại Đông Kinh\nPhật-giáo và nền văn minh\n  kỹ nghệ thế giới\nPhật giáo và nền hòa bình thế giới\n\nKinh Pháp-Cá - Thích Thiện-Siêu\nCan thong-nhat that su P.G.V.N. - Trọng-Đức\nKhoa học Duy thức - Kao-Khả Chính (dịch)\nBén do (tho) - Thiệu-dung\nQuả-háo (truyện ngắn) - Bá-Triệu\nTin-tuc.\n  </page>\n  ',
 '<page page="3">\n  </page>\n  ',
 '<page page="4">\n    NHỮNG ĐẠI-LỄ\n    BUDDHA JAYANTI\nLIÊN-TIẾP trong mấy năm nay, lễ Buldha Jayanti, lễ kỷ\nniệm 2.joo năm ngày đầu sanh đức Phật Thích-Ca, đã được\nlong trọng cử hành tại các nước Phật-giáo: Tích lan, Diện-điện,\nẤn-độ, Hồi-quốc, Népal, Cao Mên... những lễ ấy đã được tổ.\nchức thành những lễ có tánh cách quốc tế với sự tham dự của các\nphái đoàn Phật-tử trên thế\ngiới. 

In [20]:
print(system_message_clean)


- You are a meticulous and consistent world expert at cleaning OCR-generated Vietnamese text. 
- You are cleaning text from a 1950's Buddhist Journal. 
IMPORTANT: You will receive one page of text listed by lines: do not modify the line structure.
- You will minimally modify the text to fix any OCR errors.
- You will add <section>, <title>, <author>, <p>, <li>, <ul>, <ol> tags (only) where appropriate in text, based on text layout and content. 
- Do not remove any content from the main body of the text.   
- Use patterns in the text blocks (given by page) to infer patterns in the text.
- Use the semantic meaning of the text to infer corrections—but make no semantic changes. 
- Add diacritical marks if they are missing or clearly inaccurate. 
- Do not change any proper names, except to add missing diacritical marks if the context is clear.  
IMPORTANT: Output the corrected text only with no comments or additional formatting marks of any kind.



In [17]:
# generate all cleaning batch files using pages:

processed_journals = "../../processed_journal_data"
batch_ouput_prefix = "./journal_cleaning_batches/"

for path in Path(processed_journals).iterdir():
    if path.is_dir():
        journal_name = path.name
        for subpath in Path(path).iterdir():
            regex = re.compile(r"^full_OCR_.*\.xml")
            if subpath.is_file() and regex.search(subpath.name):
                print(subpath.name)
                try:
                    ocr_text = get_text_from_file(subpath.name, path)
                    print(f"{ocr_text[:90]}...")
                    chunks = split_xml_pages(ocr_text)
                    clean_message_seq = generate_messages(system_message_clean, user_wrap_function_clean, chunks)
                    create_jsonl_file_for_batch(clean_message_seq, batch_ouput_prefix + "clean_batch_" + journal_name + ".jsonl")
                except Exception as e:
                    print(f"{e}\nfailed.. \nContinuing.")
                    

full_OCR_text_phat-giao-viet-nam-1956-14.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-14</tit...
full_OCR_text_phat-giao-viet-nam-1956-13.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-13</tit...
full_OCR_text_phat-giao-viet-nam-1956-22.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-22</tit...
full_OCR_text_phat-giao-viet-nam-1956-23.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-23</tit...
full_OCR_text_phat-giao-viet-nam-1956-24.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-24</tit...
full_OCR_text_phat-giao-viet-nam-1956-12.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-12</tit...
full_OCR_text_phat-giao-viet-nam-1956-15.xml
<?xml version='1.0' encoding='UTF-8'?>
<document>
  <title>phat-giao-viet-nam-1956-15</tit...
full_OCR_text_phat-giao-vie

In [21]:
batch_client = set_api_client()

In [22]:
batch_job_dir = "./journal_cleaning_batches"
batch_files = os.listdir(batch_job_dir)
batch_files


['clean_batch_phat-giao-viet-nam-1956-12.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-14.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-09-10.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-16.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-11.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-25-26.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-13.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-17-18.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-15.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-28.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-24.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-19.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-02.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-22.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-04.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-03.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-27.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-01.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-05-06.jsonl',
 'clean_batch_phat-giao-viet-nam-1956-23.jsonl',
 'clean_

In [23]:
file_path = os.path.join(batch_job_dir, 'clean_batch_phat-giao-viet-nam-1956-28.jsonl')
file_path

'./journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-28.jsonl'

## completed batches:
10, 25-26

In [24]:
batch28 = start_batch(file_path)

Batch Initiated: 11-20-2024 11:56:20 PST | ./journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-28.jsonl


In [14]:
# file_path

In [15]:
# batch_27 = start_batch(file_path)

In [16]:
# batch_05_06 = start_batch(file_path)

In [25]:
get_active_batches()

[{'id': 'batch_673e3ee6d2a08190acf046bbe548876a',
  'status': 'in_progress',
  'created_at': 1732132582}]

In [26]:
completed = get_completed_batches()

In [27]:
completed

[{'id': 'batch_673e3ee6d2a08190acf046bbe548876a',
  'status': 'completed',
  'created_at': 1732132582,
  'output_file_id': 'file-YYyH6iRziAM8IrZle84Krzqu',
  'metadata': {'description': '11-20-2024 11:56:20 PST | ./journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-28.jsonl',
   'basename': 'clean_batch_phat-giao-viet-nam-1956-28'}},
 {'id': 'batch_673a9da47afc8190a188040a9bde09c1',
  'status': 'completed',
  'created_at': 1731894692,
  'output_file_id': 'file-0AADMzHMN28V68wFiOZoBhHV',
  'metadata': {'description': '11-17-2024 17:51:30 PST | ./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-25-26.jsonl',
   'basename': 'translate_batch_phat-giao-viet-nam-1956-25-26'}},
 {'id': 'batch_673a8f19dad881908822d978dcaadc76',
  'status': 'completed',
  'created_at': 1731890970,
  'output_file_id': 'file-utIiWGSymxTXYOuvJ8QgBM66',
  'metadata': {'description': '11-17-2024 16:49:27 PST | ./journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-25-26.jsonl',
   

In [28]:
cleaned_data = get_batch_response(completed[0]['id'])

In [29]:
len(cleaned_data)

51

In [31]:
print(cleaned_data[3])

<page page="4">
    <section>
        <title>NHỮNG ĐẠI-LỄ</title>
        <title>BUDDHA JAYANTI</title>
        <p>LIÊN-TIẾP trong mấy năm nay, lễ Buddha Jayanti, lễ kỷ niệm 2.500 năm ngày đản sanh Đức Phật Thích-Ca, đã được long trọng cử hành tại các nước Phật-giáo: Tích Lan, Diện-Điện, Ấn-Độ, Hồi-Quốc, Nepal, Cao Mên... Những lễ ấy đã được tổ chức thành những lễ có tính cách quốc tế với sự tham dự của các phái đoàn Phật-tử trên thế giới. Chúng ta chưa quên vẻ vĩ đại, huy hoàng của những lễ ấy ở Ấn Độ, Diện-Điện, Hồi-Quốc, thì năm nay nước Nhật lại tiếp tục tô thêm đậm nét rực rỡ cho ngày ra đời của Đức Phật bằng những buổi đại lễ từ ngày 27 đến 31 tháng 3 dương lịch vừa qua.</p>
        <p>Cũng như các cuộc lễ Buddha Jayanti đã cử hành ở các nước Phật-giáo, nước Nhật đã nhân dịp có các đại biểu Phật tử thế giới đến dự lễ để mở những hội nghị trao đổi quan điểm về giáo lý Đạo Phật đối với đời sống hiện đại của nhân loại. Không những chỉ các nước Phật giáo được mời đến dự, mà những tư 

In [37]:
# deprectated: now <page> markers stay in cleaned text.
# def join_pages(data):
#     result = ["<document>"]
#     for i, page in enumerate(data):
#         result.append(f"<page page=\"{i+1}\">\n")
#         result.append(escape(page))
#         result.append("\n</page>\n")
#     result.append("</document>")

#     return "".join(result)

def join_pages(data):
    result = ["<document>"]
    result = result + data
    result.append("</document>")

    return "\n".join(result)


In [38]:
cleaned_data

['<page page="1">\n    <title>PHẬT GIÁO VIỆT-NAM NGUYỆT SAN</title>\n    <p>SỐ 28 THÁNG 3 NĂM KỶ HỢI</p>\n    <p>THÁNG 4 29</p>\n    <p>BỘ CÔNG HỘI PHẬT GIÁO CHỊU NẰM XUẤT LÀM</p>\n</page>',
 '<page page="2">\n    <section>\n        <title>MỤC LỤC</title>\n        <ul>\n            <li>Những ngày đại-lễ Buddha Jayanti - P.G.V.N</li>\n            <li>Thông điệp của phái đoàn - P.G.V.N. tại Đông Kinh</li>\n            <li>Phật-giáo và nền văn minh kỹ nghệ thế giới</li>\n            <li>Phật giáo và nền hòa bình thế giới</li>\n            <li>Kinh Pháp-Cú - Thích Thiện-Siêu</li>\n            <li>Cần thống nhất thật sự P.G.V.N. - Trọng-Đức</li>\n            <li>Khoa học Duy thức - Kao-Khả Chính (dịch)</li>\n            <li>Bến mơ (thơ) - Thiệu-Dung</li>\n            <li>Quả-háo (truyện ngắn) - Bá-Triệu</li>\n            <li>Tin tức.</li>\n        </ul>\n    </section>\n</page>',
 '<page page="3">\n</page>',
 '<page page="4">\n    <section>\n        <title>NHỮNG ĐẠI-LỄ</title>\n        <tit

In [39]:
full_cleaned_text = join_pages(cleaned_data)

In [40]:
print(full_cleaned_text)

<document>
<page page="1">
    <title>PHẬT GIÁO VIỆT-NAM NGUYỆT SAN</title>
    <p>SỐ 28 THÁNG 3 NĂM KỶ HỢI</p>
    <p>THÁNG 4 29</p>
    <p>BỘ CÔNG HỘI PHẬT GIÁO CHỊU NẰM XUẤT LÀM</p>
</page>
<page page="2">
    <section>
        <title>MỤC LỤC</title>
        <ul>
            <li>Những ngày đại-lễ Buddha Jayanti - P.G.V.N</li>
            <li>Thông điệp của phái đoàn - P.G.V.N. tại Đông Kinh</li>
            <li>Phật-giáo và nền văn minh kỹ nghệ thế giới</li>
            <li>Phật giáo và nền hòa bình thế giới</li>
            <li>Kinh Pháp-Cú - Thích Thiện-Siêu</li>
            <li>Cần thống nhất thật sự P.G.V.N. - Trọng-Đức</li>
            <li>Khoa học Duy thức - Kao-Khả Chính (dịch)</li>
            <li>Bến mơ (thơ) - Thiệu-Dung</li>
            <li>Quả-háo (truyện ngắn) - Bá-Triệu</li>
            <li>Tin tức.</li>
        </ul>
    </section>
</page>
<page page="3">
</page>
<page page="4">
    <section>
        <title>NHỮNG ĐẠI-LỄ</title>
        <title>BUDDHA JAYANTI</title>
  

XMLSyntaxError: Opening and ending tag mismatch: br line 98 and p, line 99, column 48 (<string>, line 99)
Offending line 99, column 47: tác phẩm của văn-sĩ Nhật Shintaro Ishihara.</p>


[]

In [ ]:
#write_text_to_file("full_cleaned_journal_28.xml", full_cleaned_text)

In [58]:
full_cleaned_text = get_text_from_file("full_cleaned_journal_28.xml")

In [34]:
token_count(full_cleaned_text)

26495

In [71]:
cleaned_sections = split_xml_pages(full_cleaned_text, page_groups=[(1, 6), (7, 17),(18, 25), (26, 30), (31, 36), (37, 37), (38, 44), (45, 51)])

In [ ]:
[token_count(x) for x in cleaned_sections]

[2148, 6219, 4406, 2697, 3009, 629, 3438, 3763]

In [ ]:
[type(x) for x in cleaned_sections]

[str, str, str, str, str, str, str, str]

In [72]:
print(cleaned_sections[0])

<page page="1">
    <title>PHẬT GIÁO VIỆT-NAM NGUYỆT SAN</title>
    <p>SỐ 28 THÁNG 3 NĂM KỶ HỢI</p>
    <p>THÁNG 4 29</p>
    <p>BỘ CÔNG HỘI PHẬT GIÁO CHỊU NẰM XUẤT LÀM</p>
</page>
<page page="2">
    <section>
        <title>MỤC LỤC</title>
        <ul>
            <li>Những ngày đại-lễ Buddha Jayanti - P.G.V.N</li>
            <li>Thông điệp của phái đoàn - P.G.V.N. tại Đông Kinh</li>
            <li>Phật-giáo và nền văn minh kỹ nghệ thế giới</li>
            <li>Phật giáo và nền hòa bình thế giới</li>
            <li>Kinh Pháp-Cú - Thích Thiện-Siêu</li>
            <li>Cần thống nhất thật sự P.G.V.N. - Trọng-Đức</li>
            <li>Khoa học Duy thức - Kao-Khả Chính (dịch)</li>
            <li>Bến mơ (thơ) - Thiệu-Dung</li>
            <li>Quả-háo (truyện ngắn) - Bá-Triệu</li>
            <li>Tin tức.</li>
        </ul>
    </section>
</page>
<page page="3">
</page>
<page page="4">
    <section>
        <title>NHỮNG ĐẠI-LỄ</title>
        <title>BUDDHA JAYANTI</title>
        <p>LI

In [ ]:
system_message_translate = """
You are translating as Thich Nhat Hanh from Vietnamese to English for your experienced students. 
The text is from 1956-1958. Use the title: "Vietnamese Buddhist Journal." 
You want experienced students to understand the material and its historical and cultural context—in particular, as it relates to the life and teaching of Thich Nhat Hanh.

- You will receive 1 section of the journal at a time. 
- Give a full English translation in the style of Thich Nhat Hanh. Precision and clarity are important.
- Format the text neatly.
- Keep pages together: each translated page must match it's original page source as pages will be studied side by side with the original Vietnamese.
- Exception: you can move a sentence if it is split between two pages.
- Pages will already have metadata tags that were generated page by page.
- Edit and adjust XML tags as needed to join page information and create clarity.
- Add additional XML tags for clarity.
- All titles and XML sections should also be translated.
- Do not make comments within the text itself. 
- Leave all names of people in original Vietnamese with diacritics. 
- Each section of the text should have its own dedicated `<footnotes>` section.
- Use footnotes generously to explain elements of Vietnamese Buddhism or Buddhism in general, elements of Vietnamese culture and history, or elements relative to the life, teachings, and practice of Thich Nhat Hanh.
- When translating complex terms (Sanskrit, Sino-Vietnamese, or French) mark these with <i></i> tags and give an explanation in the footnotes.  
- For any terms footnoted, give the original Vietnamese, Sino-Vietnamese, Sanskrit, or French in the footnote. 

Footnote Formatting:
Inline Footnote Markers:
   - In the main text, footnotes should be marked numerically in brackets, such as "[1]".
   - Enclose each inline footnote marker within a `<footnote number="n">` tag, where `n` is the footnote number.
   - Example of a footnoted paragraph:
     `<p>This paragraph has a footnote. This is a sentence in the paragraph.</p><footnote number="1">[1]</footnote>`

Footnote Section:
   - If a page has footnotes then it should have a `<footnotes>` section at the end of the page with the information for each footnote.
   - Each footnote in the `<footnotes>` section should use a `<footnote number="n">` tag, with `n` matching the inline reference.

Numbering:
   - Footnote numbers should restart at 1 for each new page of text.

Here's an example of a page containing two footnotes:

```
   <page> 
      <p>This is sentence has a footnote.<footnote number="1">[1]</footnote> <p>
      <p>Here is another statement that needs a footnote.<footnote number="2">[2]</footnote></p>`
      <p>The rest of the page follows ... </p>   
      ...
      <footnotes>
         <footnote number="1">This is the information for the first footnote</footnote>`
         <footnote number="2">This is the information for the second footnote</footnote>`
      </footnotes>
   </page>
```
"""

In [80]:
token_count(system_message_translate)

718

In [81]:
user_message_string_translate = """{text}"""

In [82]:
def user_wrap_function_translate(text_block):
    return user_message_string_translate.format(text=text_block)

In [84]:
translation_message_seq = generate_messages(system_message_translate, user_wrap_function_translate, cleaned_sections)

In [90]:
processed_journals = "../../processed_journal_data"
batch_ouput_prefix = "./journal_translate_batches/"
journal_name = "phat-giao-viet-nam-1956-28"

In [91]:
create_jsonl_file_for_batch(translation_message_seq, batch_ouput_prefix + "translate_batch_" + journal_name + ".jsonl")

'./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-28.jsonl'

In [92]:
batch_job_dir = "./journal_translate_batches"
batch_path = os.path.join(batch_job_dir, 'translate_batch_phat-giao-viet-nam-1956-28.jsonl')
batch_path

'./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-28.jsonl'

In [93]:
tx_batch = start_batch(batch_path)
tx_batch

Batch Initiated: 11-20-2024 13:02:33 PST | ./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-28.jsonl


Batch(id='batch_673e4e6aecb88190adabba0b015feaac', completion_window='24h', created_at=1732136554, endpoint='/v1/chat/completions', input_file_id='file-hKqQTQvqftmkW4WsqiaIyKa1', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1732222954, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': '11-20-2024 13:02:33 PST | ./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-28.jsonl', 'basename': 'translate_batch_phat-giao-viet-nam-1956-28'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [95]:
get_active_batches()

[]

In [96]:
completed = get_completed_batches()

In [97]:
completed

[{'id': 'batch_673e4e6aecb88190adabba0b015feaac',
  'status': 'completed',
  'created_at': 1732136554,
  'output_file_id': 'file-nIfO0fEwX4P6ACm298dyuLP0',
  'metadata': {'description': '11-20-2024 13:02:33 PST | ./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-28.jsonl',
   'basename': 'translate_batch_phat-giao-viet-nam-1956-28'}},
 {'id': 'batch_673e3ee6d2a08190acf046bbe548876a',
  'status': 'completed',
  'created_at': 1732132582,
  'output_file_id': 'file-YYyH6iRziAM8IrZle84Krzqu',
  'metadata': {'description': '11-20-2024 11:56:20 PST | ./journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-28.jsonl',
   'basename': 'clean_batch_phat-giao-viet-nam-1956-28'}},
 {'id': 'batch_673a9da47afc8190a188040a9bde09c1',
  'status': 'completed',
  'created_at': 1731894692,
  'output_file_id': 'file-0AADMzHMN28V68wFiOZoBhHV',
  'metadata': {'description': '11-17-2024 17:51:30 PST | ./journal_translate_batches/translate_batch_phat-giao-viet-nam-1956-25-26.jsonl',
   '

In [98]:
translated_data = get_batch_response(completed[0]['id'])

In [99]:
len(translated_data)

8

In [101]:
full_translated_text = join_pages(translated_data)

In [102]:
print(full_translated_text)

<document>
<page page="1">
    <title>Vietnamese Buddhist Journal</title>
    <p>Issue 28, March Year of the Pig (Kỷ Hợi)</p>
    <p>April 29</p>
    <p>Department of Buddhist Affairs Responsible for Publishing</p>
</page>
<page page="2">
    <section>
        <title>Table of Contents</title>
        <ul>
            <li>The Great Days of Buddha Jayanti - V.B.J.</li>
            <li>Message from the Delegation - V.B.J. in Tokyo</li>
            <li>Buddhism and The World's Industrial Civilization</li>
            <li>Buddhism and World Peace</li>
            <li>The Dhammapada - Thích Thiện-Siêu</li>
            <li>The Need for True Unification of V.B.J. - Trọng-Đức</li>
            <li>Science of Consciousness Only - Kao-Khả Chính (translation)</li>
            <li>The Dreaming Harbor (poetry) - Thiệu-Dung</li>
            <li>The Fruit of Benevolence (short story) - Bá-Triệu</li>
            <li>News.</li>
        </ul>
    </section>
</page>
<page page="3">
</page>
<page page="4">


In [103]:
token_count(full_translated_text)

19623

In [104]:
write_text_to_file("full_tx_phat-giao-viet-nam-1956-28.xml", full_translated_text)